In [3]:
from mumax_helper import *

from scipy.signal import find_peaks

In [4]:
# NUMERICAL PARAMETERS RELEVANT FOR THE SPECTRUM ANALYSIS
fmax = 5e9        # maximum frequency (in Hz) of the sinc pulse
T = 50e-9        # simulation time (longer -> better frequency resolution)
dt = 1/(2*fmax)  # the sample time (Nyquist theorem taken into account)
side_length = 300e-9  # diameter of the circle domain
fdrive = 750e8
Lx = 200
Ly = 128
Lz = 1


# Note that this is a format string, this means that the statements inside the
# curly brackets get evaluated by python. In this way, we insert the values of
# the variables above in the script.

def simulation(field, freq):

    script = f"""
    setgridsize({Lx},{Ly},{Lz})
    setcellsize(1e-9,1e-9,1e-9)

    domain := rect({Lx},{Ly})
    tooth_length := 50e-9
    box1 := square(tooth_length).rotz(pi/4).transl(0,{Ly}*1e-9/2,0)
    box2 := square(tooth_length).rotz(pi/4).transl(-{Lx}/4*1e-9,{Ly}*1e-9/2,0)
    box3 := square(tooth_length).rotz(pi/4).transl({Lx}/4*1e-9,{Ly}*1e-9/2,0)
    box4 := square(tooth_length).rotz(pi/4).transl(-{Lx}/2*1e-9,{Ly}*1e-9/2,0)
    box5 := square(tooth_length).rotz(pi/4).transl({Lx}/2*1e-9,{Ly}*1e-9/2,0)

    setgeom(domain.sub(box1).sub(box2).sub(box3).sub(box4).sub(box5))

    setpbc(3,0,0)

    Msat = 1e6
    Aex = 10e-12
    Dind = 2.25e-3
    Ku1 = 1e6
    AnisU = vector(0,0,1)
    alpha = 0.001

    B_ext = vector(0, 0, {field} * sinc( 2*pi*{freq}*t)) 

    tableAdd(B_ext)
    tableAdd(ext_bubblespeed)

    m = NeelSkyrmion(1,-1)

    minimize()
    autosave(m,{dt})
    tableautosave({dt})
    run({T})
    """

    return script

run_mumax3(simulation(.0001, fmax), "Zigzag boarder excite")


In [28]:
# NUMERICAL PARAMETERS RELEVANT FOR THE SPECTRUM ANALYSIS
fmax = 5e9        # maximum frequency (in Hz) of the sinc pulse
T = 100e-9        # simulation time (longer -> better frequency resolution)
dt = 1/(2*fmax)  # the sample time (Nyquist theorem taken into account)
side_length = 300e-9  # diameter of the circle domain
fdrive = 750e8
Lx = 180
Ly = 100
Lz = 1

cs = 1e-9


# Note that this is a format string, this means that the statements inside the
# curly brackets get evaluated by python. In this way, we insert the values of
# the variables above in the script.

def simulation(field, freq):

    script = f"""
    setgridsize({Lx},{Ly},{Lz})
    setcellsize({cs},{cs},{cs})

    domain := rect({Lx},{Ly})
    tooth_length := 50e-9
    box1 := square(tooth_length).rotz(pi/4).transl(0,{Ly}*1e-9/2,0)
    box2 := square(tooth_length).rotz(pi/4).transl(-{Lx}/4*1e-9,{Ly}*1e-9/2,0)
    box3 := square(tooth_length).rotz(pi/4).transl({Lx}/4*1e-9,{Ly}*1e-9/2,0)
    box4 := square(tooth_length).rotz(pi/4).transl(-{Lx}/2*1e-9,{Ly}*1e-9/2,0)
    box5 := square(tooth_length).rotz(pi/4).transl({Lx}/2*1e-9,{Ly}*1e-9/2,0)

    offset := -tooth_length/2
    box6 := square(tooth_length).rotz(pi/4).transl(({Lx}/2*1e-9 + offset),-{Ly}*1e-9/2,0)
    box7 := square(tooth_length).rotz(pi/4).transl(-({Lx}/4*1e-9 + offset),-{Ly}*1e-9/2,0)
    box8 := square(tooth_length).rotz(pi/4).transl(({Lx}/4*1e-9 + offset),-{Ly}*1e-9/2,0)
    box9 := square(tooth_length).rotz(pi/4).transl(-({Lx}/2*1e-9 + offset),-{Ly}*1e-9/2,0)


    setgeom(domain.sub(box1).sub(box2).sub(box3).sub(box4).sub(box5))

    setpbc(3,0,0)

    Msat = 1e6
    Aex = 10e-12
    Dind = 2.25e-3
    Ku1 = 1e6
    AnisU = vector(0,0,1)
    alpha = 0.001

    // B_ext = vector(0, 0, {field} * sinc( 2*pi*{freq}*t)) 
    B_ext = vector(0, 0, {field} * sin( 2*pi*{freq}*t)) 

    tableAdd(B_ext)
    tableAdd(ext_bubblespeed)

    m = NeelSkyrmion(1,-1)

    minimize()
    autosave(m,{dt})
    tableautosave({dt})
    run({T})
    """

    return script


run_mumax3(simulation(.02, fmax), "Zigzag boarder")


In [6]:
freq_list = np.linspace(5e8,1e10,15)

for freq in freq_list:
    title = f"Zigzag boarder f={freq}"
    run_mumax3(simulation(.02,fmax), title)